In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle_gskkuedu.json to kaggle_gskkuedu.json


{'kaggle_gskkuedu.json': b'{"username":"jihwaa","key":"851c1a6946283e12e1abf74d51dfaf51"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

ref                                             deadline             category            reward  teamCount  userHasEntered  
----------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                    2030-07-01 23:59:00  Getting Started     Prizes         69           False  
gan-getting-started                             2030-07-01 23:59:00  Getting Started     Prizes         94           False  
store-sales-time-series-forecasting             2030-06-30 23:59:00  Getting Started  Knowledge        856           False  
tpu-getting-started                             2030-06-03 23:59:00  Getting Started  Knowledge        146           False  
digit-recognizer                                2030-01-01 00:00:00  Getting Started  Knowledge       1247           False  
titanic                                         2030-01-01 00:00:00  Getting Started  Knowledge      14114            True  


In [3]:
!kaggle competitions download -c skku-dnn-hw5

  0% 0.00/1.03M [00:00<?, ?B/s]
100% 1.03M/1.03M [00:00<00:00, 151MB/s]


In [4]:
!unzip skku-dnn-hw5.zip

Archive:  skku-dnn-hw5.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 182 kB 87.3 MB/s 
     |████████████████████████████████| 7.6 MB 94.8 MB/s 


In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [7]:
max_len = 50
pad_idx, eos_idx, bos_idx, unk_idx = 0, 1, 2, 3

""" Tokenizer """
class WordTokenizer:
    def __init__(self, max_len):
        self.word2idx = {'[PAD]': pad_idx, '[EOS]': eos_idx, '[BOS]': bos_idx, '[UNK]': unk_idx}
        self.idx2word = {pad_idx: '[PAD]', eos_idx: '[EOS]', bos_idx: '[BOS]', unk_idx: '[UNK]'}
        self.max_len = max_len
        
    def __len__(self):
        return len(self.word2idx)
    
    def encode(self, sents):
        max_len = min(max([len(sent.split()) for sent in sents]), self.max_len) + 2 # Consider [BOS], [EOS] tokens for PLM
        
        idx, padding_mask = [], []
        for sent in sents:
            temp, temp_mask = self.sent_to_idx(sent)
            # Truncation
            if len(temp) > max_len:
                temp = temp[:max_len-1] + [self.word_to_idx('[EOS]')]
            temp_mask = temp_mask[:max_len]
        
            #Padding
            if len(temp) < max_len:
                for i in range(max_len-len(temp)):
                    temp += [self.word_to_idx('[PAD]')]
                    temp_mask.append(0)
                    
            idx.append(temp)
            padding_mask.append(temp_mask)
        
        return idx, padding_mask
    
    def decode(self, sent):
        return self.idx_to_sent(sent)
    
    def sent_update(self, sent):
        for w in sent.split(' '):
            self.word_update(w)
    
    def word_update(self, word):
        if not word in self.word2idx.keys():
            self.word2idx[word] = len(self.word2idx)
            self.idx2word[len(self.idx2word)] = word
            
    def word_to_idx(self, word):
        try:
            return self.word2idx[word]
        except:
            return self.word2idx['[UNK]']
        
    def idx_to_word(self, idx):
        try:
            return self.idx2word[idx]
        except:
            return self.idx2word[unk_idx]
        
    def sent_to_idx(self, sent, no_bos_eos=False):
        idx = [self.word_to_idx('[BOS]')]

        for w in sent.split(' '):
            idx.append(self.word_to_idx(w))

        idx.append(self.word_to_idx('[EOS]'))
        # tokenized sentence, tokenized sentence attention
        return idx, [1]*len(idx)

    def idx_to_sent(self, idx):
        sent = []
        for i in idx:
            if i == self.word_to_idx('[PAD]') or i == self.word_to_idx('[BOS]'):
                continue
            
            elif i == self.word_to_idx('[EOS]'):
                break

            sent.append(self.idx_to_word(i))

        return ' '.join(sent)
    
""" Get numeric answer """
def is_num(w):
    if len(w) == 0:
        return False

    elif len(w) == 1:
        if w.isdigit():
            return True
        
        else:
            return False
    
    elif w[0] == '.' and w[1].isdigit(): # mawps .01
        return True

    elif '/' in w:
        if w.split('/')[0].isdigit() and w.split('/')[1].isdigit():
            return True

    # SVAMP: 1-year
    return (w[0].isdigit() and w[1] != '-') or ((w[0]=='-' and len(w) != 1) and w[1].isdigit())

def convert_post_to_in(sent):
    OPERATOR_DICT = {
        '[OP_ADD]': '+', '[OP_SUB]': '-', '[OP_MUL]': '*', '[OP_DIV]': '/'
    }
    
    s = []
    for i in sent.split():
        if not i.startswith('[OP'):
            s.insert(0, i)
        else:
            op1 = s[0]
            if op1.startswith('[C_'):
                op1 = op1[3:-1]
            s.pop(0)
            
            op2 = s[0]
            if op2.startswith('[C_]'):
                op2 = op2[3:-1]
            s.pop(0)
            
            s.insert(0, "( " + op2 + ' ' + OPERATOR_DICT[i] + ' ' + op1 + ' )')
            
    return s[0]

# Postfix -> Infix나 eval(Infix equation)에서 오류가 날 경우 -1 return
def get_answer(sent):
    try:
        new_sent = convert_post_to_in(sent) # Postfix -> Infix
        try:
            answer = eval(new_sent)
        except:
            answer = -1 # Exception handling
    except:
        answer = -1
    return f'{answer:.2f}'

In [8]:
# Custom Dataset
class MWPDataset(Dataset):
    def __init__(self, x, y, ans):
        self.x = x
        self.y = y
        self.ans = ans
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return {
            'problem': self.x[idx],
            'solution': self.y[idx] if not self.y is None else None,
            'answer': self.ans[idx] if not self.ans is None else None
        }

# Custom collate fn function
def collate_fn(batch):
    problem_list, solution_list, answer_list, n_list = [], [], [], []
    for idx, b in enumerate(batch):
        problem_list.append(b['problem'])
        solution_list.append(b['solution'])
        answer_list.append(b['answer'])
        
        # N dictionary save
        value_dict = dict()
        for w in b['problem'].split():
            if is_num(w):
                num, subword = '', ''
                for i, c in enumerate(w):
                    if c in '1234567890./-':
                        num += c
                    elif c in [',']:
                        continue
                    else:
                        subword = w[i:]
                if num[-1] in '-.':
                    num = num[:-1]
                
                n_idx = ''
                for k, v in value_dict.items():
                    if v == num:
                        n_idx = k
                
                if n_idx == '':
                    n_idx = '[N_' + str(len(value_dict)) + ']'
                    value_dict[n_idx] = num
                    
        n_list.append(value_dict)
                
    tokenized_problem = encoder_tokenizer(problem_list, padding=True, truncation=True, return_tensors='np')
    problem, problem_attn_mask = tokenized_problem['input_ids'], tokenized_problem['attention_mask']
    
    if not solution_list[0] is None:
        solution, solution_attn_mask = decoder_tokenizer.encode(solution_list)

        batch_dict = {
            'problem': torch.tensor(problem, device=device),
            'problem_attn_mask': torch.tensor(problem_attn_mask, device=device),
            'solution': torch.tensor(solution, device=device),
            'solution_attn_mask': torch.tensor(solution_attn_mask, device=device),
        }
    else:
        batch_dict = {
            'problem': torch.tensor(problem, device=device),
            'problem_attn_mask': torch.tensor(problem_attn_mask, device=device),
            'solution': None,
            'solution_attn_mask': None,
            'n_list': n_list
        }
    
    return batch_dict

In [9]:
# Read train data
train = pd.read_csv('train.csv')
train_x = train.problem
train_y = train.solution
train_ans = train.answer

# Read test data
test = pd.read_csv('test.csv')
test_x = test.problem

# Pre-trained language model path in huggingface
pretrained_path = 'bert-base-uncased'

# Encoder pre-trained tokenizer
encoder_tokenizer = AutoTokenizer.from_pretrained(pretrained_path, model_max_length=512)
# Update special vocabs
special_vocab = set()
for sent in train_x:
    for w in sent.split():
        if w.startswith('[N'):
            special_vocab.add(w)
encoder_tokenizer.add_special_tokens({"additional_special_tokens": list(special_vocab)})

# Decoder word tokenizer
decoder_tokenizer = WordTokenizer(max_len=max_len)
for sent in train_y:
    for w in sent.split():
        decoder_tokenizer.sent_update(sent)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
# Traning arguments
train_args = {
    'batch_size': 64,
    'lr': 1e-4,
    'weight_decay': 1e-2,
    'epochs': 150,
    'teacher_forcing_ratio': 0.3
}

# Test arguments
test_args = {
    'batch_size': 64
}

In [11]:
# Make datatsets
train_dataset = MWPDataset(train_x, train_y, train_ans)
test_dataset = MWPDataset(test_x, None, None)

# Make dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=train_args['batch_size'], shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=test_args['batch_size'], shuffle=False, collate_fn=collate_fn)

In [12]:
class Baseline(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size):
        super().__init__()
        self.input_vocab_size = input_vocab_size # Input vocabulary size
        self.output_vocab_size = output_vocab_size # Output vocabulary size
        
        self.num_layers = 2 # Decoder num layers
        self.teacher_force = 0.3 # Decoder teacher forcing ratio
        
        self.encoder = AutoModel.from_pretrained(pretrained_path) # Load pre-trained model
        self.encoder.resize_token_embeddings(self.input_vocab_size) # Resize pre-trained model embedding according to input vocabulary size
        self.decoder_embedding = nn.Embedding(self.output_vocab_size, self.encoder.config.hidden_size) # Decoder embedding with hidden size of encoder
        self.decoder = nn.LSTM(input_size=self.encoder.config.hidden_size, hidden_size=self.encoder.config.hidden_size,
                               num_layers=self.num_layers, bidirectional=True, dropout=0.1) # LSTM decoder
        self.linear= nn.Linear(self.encoder.config.hidden_size*2, self.output_vocab_size)
        
        self.to(device)
        
    def forward(self, batch, epoch):
        encoder_outputs = self.encoder(input_ids=batch['problem'], attention_mask=batch['problem_attn_mask'])[0]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        decoder_embed_outputs = self.decoder_embedding(batch['solution'])
        decoder_embed_outputs = decoder_embed_outputs.permute(1, 0, 2)
        
        sol_len, batch_size, _ = decoder_embed_outputs.shape
        hidden = encoder_outputs[-1,:,:].repeat(self.num_layers*2, 1, 1) # (self.num_layers, batch_size, hidden_dim)
        cell = encoder_outputs[-1,:,:].repeat(self.num_layers*2, 1, 1) # (self.num_layers, batch_size, hidden_dim)
        
        outputs = torch.zeros(sol_len-1, batch_size, self.output_vocab_size, device=device)
        inputs = decoder_embed_outputs[0, :, :].unsqueeze(0) # [BOS]
        
        for t in range(1, sol_len):
            output, (hidden, cell) = self.decoder(inputs, (hidden, cell))
            prediction = self.linear(output.squeeze(0))
            
            outputs[t-1] = prediction
#             teacher forcing ratio decay
            teacher_force = random.random() < train_args['teacher_forcing_ratio'] - ((0.3- 0.1) / 150) * epoch
            inputs = decoder_embed_outputs[t,:,:] if teacher_force else self.decoder_embedding(prediction.argmax(1))
            inputs = inputs.unsqueeze(0) # (1, batch_size, hidden_dim)
            
        logits = outputs.permute(1, 0, 2) # (batch_size, sol_len, output_vocab_size)
        
        return logits
    
    def evaluate(self, batch):
        encoder_outputs = self.encoder(input_ids=batch['problem'], attention_mask=batch['problem_attn_mask'])[0]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        hidden = encoder_outputs[-1,:,:].repeat(self.num_layers*2, 1, 1)
        cell = encoder_outputs[-1,:,:].repeat(self.num_layers*2, 1, 1)
        
        batch_size = encoder_outputs.shape[1]
        pred = [[bos_idx] for _ in range(batch_size)]
        max_len = 50
        
        # Greedy
        for i in range(max_len):
            cur_pred = torch.tensor(pred, device=device) if i == 0 else cur_pred.unsqueeze(1)
            inputs = self.decoder_embedding(cur_pred)
            inputs = inputs.permute(1, 0, 2)
            
            output, (hidden, cell) = self.decoder(inputs, (hidden, cell))
            prediction = self.linear(output.squeeze(0))
            
            cur_pred = prediction.argmax(1)
            
            for b in range(batch_size):
                pred[b].append(cur_pred[b].item())
                
        for b in range(batch_size):
            pred[b] = pred[b][1:]
        
        return pred

In [13]:
# model
model = Baseline(len(encoder_tokenizer), len(decoder_tokenizer))

# optimizer
optimizer = optim.AdamW(model.parameters(), lr=train_args['lr'], weight_decay=train_args['weight_decay'])


scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,\
			T_0=40, T_mult=1, eta_min=(1e-6) * (1/2), last_epoch=-1)

# loss function
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Train
for epoch in range(train_args['epochs']):
    epoch_loss = 0.0
    
    pbar = tqdm(train_dataloader, desc="train", dynamic_ncols=True)
    for batch_idx, batch in enumerate(pbar):
        logits = model(batch,epoch)
        
        batch_tgt = batch['solution'][:, 1:] # solutions, Remove [BOS]
        
        batch_loss = criterion(logits.reshape(-1, logits.shape[-1]), batch_tgt.reshape(-1))
        
        optimizer.zero_grad()
        
        batch_loss.backward()
        epoch_loss += batch_loss.item()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
        optimizer.step()
        
        pbar.set_postfix_str(f"loss: {batch_loss.item():.4f}")
    
    print(f'[Epoch {epoch}] Loss: {epoch_loss/len(train_dataloader):.4f}')
    scheduler.step()
    test_results = []

    # Test
    with torch.no_grad():
        pbar = tqdm(test_dataloader, desc="test", dynamic_ncols=True)
        for batch_idx, batch in enumerate(pbar):
            preds = model.evaluate(batch)

            for b_idx, pred in enumerate(preds):
                decoded_pred = decoder_tokenizer.decode(pred)

                numeric_pred = []
                for w in decoded_pred.split():
                    if w.startswith('[N_'):
                        try:
                            numeric_pred.append(batch['n_list'][b_idx][w])
                        except:
                            numeric_pred.append(w)
                    elif w.startswith('[C_'):
                        numeric_pred.append(w[3:-1])
                    else:
                        numeric_pred.append(w)
                numeric_pred = ' '.join(numeric_pred)

                answer = get_answer(numeric_pred)

                test_results.append(answer)
                
    submissions = pd.DataFrame({'id': test.id, 'answer': test_results})
    submissions.to_csv(f'submission_{epoch}.csv', index=False)

train: 100%|██████████| 94/94 [00:28<00:00,  3.34it/s, loss: 1.9147]


[Epoch 0] Loss: 2.2338


train: 100%|██████████| 94/94 [00:27<00:00,  3.43it/s, loss: 1.6052]


[Epoch 1] Loss: 1.8104


train: 100%|██████████| 94/94 [00:27<00:00,  3.42it/s, loss: 1.7464]


...

train: 100%|██████████| 94/94 [00:27<00:00,  3.46it/s, loss: 0.4770]


[Epoch 14] Loss: 0.6061


train: 100%|██████████| 94/94 [00:27<00:00,  3.44it/s, loss: 0.5361]


[Epoch 15] Loss: 0.5821


test:  24%|██▎       | 37/157 [00:05<00:19,  6.04it/s]

In [ ]:
test_results = []

# Test
with torch.no_grad():
    pbar = tqdm(test_dataloader, desc="test", dynamic_ncols=True)
    for batch_idx, batch in enumerate(pbar):
        preds = model.evaluate(batch)

        for b_idx, pred in enumerate(preds):
            decoded_pred = decoder_tokenizer.decode(pred)

            numeric_pred = []
            for w in decoded_pred.split():
                if w.startswith('[N_'):
                    try:
                        numeric_pred.append(batch['n_list'][b_idx][w])
                    except:
                        numeric_pred.append(w)
                elif w.startswith('[C_'):
                    numeric_pred.append(w[3:-1])
                else:
                    numeric_pred.append(w)
            numeric_pred = ' '.join(numeric_pred)

            answer = get_answer(numeric_pred)

            test_results.append(answer)

submissions = pd.DataFrame({'id': test.id, 'answer': test_results})
submissions.to_csv(f'submission.csv', index=False)